## Imports

In [ ]:
from glotaran.io import load_scheme

# from glotaran.project.scheme import Scheme
from glotaran.io import load_parameters
from glotaran.io import load_dataset


In [ ]:
from pyglotaran_extras import plot_data_overview
from pyglotaran_extras import plot_overview
from pyglotaran_extras.compat import convert


## Load data files

In [ ]:
experiment_data = {
    "GFPppH2O": "data/simgfph2oa.ascii",
}

plot_data_overview(experiment_data["GFPppH2O"], linlog=True, linthresh=1, cmap="seismic");


In [ ]:
parameters = load_parameters("models/parameters_sequential2.yml", format_name="yml")
parameters


In [ ]:
scheme = load_scheme("models/model_sequential_rewrite2.yml", format_name="yml")


In [ ]:
# augment data to scheme
scheme.load_data(
    {
        "GFPppH2O": load_dataset("data/simgfph2oa.ascii"),
        "GFPpdpH2O": load_dataset("data/simgfph2ob.ascii"),
    }
)


## Optimization

In [ ]:
result = scheme.optimize(parameters, maximum_number_function_evaluations=11)


In [ ]:
result.optimized_parameters


In [ ]:
# result.data["GFPppH2O"].lifetime_decay
# AttributeError: 'Dataset' object has no attribute 'lifetime_decay'


## Plotting

### Plot concentrations and spectra

In [ ]:
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas, plot_norm_sas
import matplotlib.pyplot as plt


def plot_concentration_and_spectra(result_dataset):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True)
    plot_sas(result_dataset, axes[1])
    plot_norm_sas(result_dataset, axes[2])
    return fig, axes


In [ ]:
res1_fast = result.optimization_results["GFPppH2O"].elements["fast"]
fig, axes = plot_concentration_and_spectra(res1_fast)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[0].annotate("PP", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

res2_fast = result.optimization_results["GFPpdpH2O"].elements["fast"]
fig, axes = plot_concentration_and_spectra(res2_fast)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[0].annotate("PDP", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)

#### Analyze coherent_artifact spectra

In [ ]:
res1_artifact = result.optimization_results["GFPppH2O"].elements["artifact"]
res1_artifact.dims
# join and print as string
print(f"Dimensions: {', '.join(res1_artifact.dims)}")

In [ ]:
result

In [ ]:
# Before: res1_artifact.coherent_artifact_associated_amplitudes_artifact
#    Now: result.optimization_results["GFPppH2O"].elements["artifact"].amplitudes
res1_artifact.amplitudes

In [ ]:
data_array = result.optimization_results["GFPpdpH2O"].elements["artifact"].concentrations
data_array.coords

In [ ]:
data_array = result.optimization_results["GFPpdpH2O"].elements["artifact"].concentrations

import matplotlib.pyplot as plt

# Assuming `data_array` is your DataArray
plt.plot(data_array.coords["spectral"], data_array.to_numpy().T[0][1])
# plt.plot(data_array.coords["spectral"], data_array.values[0][1])  # is all nan
plt.xlabel("Spectral")
plt.ylabel("Value")
plt.title("2D Line Plot")
plt.show()

data_array.values[0][1][0:10]

f" data.scale should not be '1.0', but estimated from the data. It's currently: {result.data['GFPpdpH2O'].scale}"


### Plot traces

In [ ]:
from pyglotaran_extras.plotting.plot_traces import (
    plot_fitted_traces,
    select_plot_wavelengths,
)
from pyglotaran_extras.plotting.style import PlotStyle

wavelengths = select_plot_wavelengths(
    convert(result.data["GFPppH2O"]), equidistant_wavelengths=False, axes_shape=(4, 3)
)
fig3tr, axes = plot_fitted_traces(
    convert(result),
    wavelengths,
    axes_shape=(4, 3),
    linlog=True,
    linthresh=1,
    cycler=PlotStyle().data_cycler_solid_dashed,
)


### Plot overview

In [ ]:
fig, _ = plot_overview(convert(result.data["GFPppH2O"]), linlog=True)


In [ ]:
fig, _ = plot_overview(convert(result.data["GFPpdpH2O"]), linlog=True)
